In [1]:
# Imports
import sys
import gurobipy as gp
import numpy as np

In [2]:
# Parameters (Valus of Instance 1)
NUM_VAGOES = 2
NUM_TRECHOS = 6
NUM_ATIVIDADES = 15
NUM_TRENS = 2
P = 5 # tempo de circulação de um vagão j em um trecho q
M = 2880
VAGOES_TREM_SAIDA = [1,1] #[2,2]
VAGOES_TREM_CHEGADA = [0,0] #[1,1]

In [3]:
# Generate AMV
# AMV[q,s] = 1 se o trecho q é ligado ao trecho s (q vizinho de s ou existe AMV entre q e s); 0 caso contrário
AMV = [[1,1,0,0,0,0],
       [1,1,1,0,1,0],
       [0,1,1,1,0,0],
       [0,0,1,1,0,0],
       [0,1,0,0,1,1],
       [0,0,0,0,1,1]]

In [4]:
# Generate PSA
# PSA[j] = trecho do pátio onde o vagão j deverá estar posicionado para que o trem t ∈ Tβ |j ∈ t possa sair
PSA = [1,2]

In [5]:
# Generate PCH
# PCH[j] = trecho do pátio onde o vagão j estará posicionado ao receber o trem t ∈ Tα|j ∈ t
PCH = [2,1]

In [6]:
# Generate Release
# RELEASE[j] = instante mais cedo que o vagão j está disponível para entrar no pátio (release date), ou seja, horário de chegada do trem t ∈ Tα|j ∈ t
RELEASE = [10,10]

In [7]:
# Simplify instances informed values
n = NUM_VAGOES
m = NUM_TRECHOS
o = NUM_ATIVIDADES
v = NUM_TRENS
t_beta = VAGOES_TREM_SAIDA
t_alfa = VAGOES_TREM_CHEGADA

In [8]:
# Initialize model
model = gp.Model("milp")

In [9]:
# Declare decision variables
var_y = model.addVars(o,n, vtype=gp.GRB.CONTINUOUS, name="y")         ## y[i,j] = instante de início da i-ésima operação do vagão j
var_z = model.addVars(o,m,n, vtype=gp.GRB.BINARY, name="z")           ## z[i,q,j] = 1 se a i-ésima operação do vagão j acontece no trecho q; 0 caso contrário
var_x = model.addVars(m,o,n,o,n, vtype=gp.GRB.BINARY, name="x")       ## x[q,i,j,k,l] = 1 se a i-ésima operação do vagão j precede a k-ésima operação do vagão l no trecho q; 0 caso contrário
var_f = model.addVars(o,n, vtype=gp.GRB.BINARY, name="f")             ## f[i,j] = 1 se a i-ésima operação do vagão j é a última, antes da partida do trem t ∈ Tβ |j ∈ t; 0 caso contrário
var_c = model.addVars(n, vtype=gp.GRB.CONTINUOUS, name="c")           ## C[j] = instante de conclusão do vagão j, horário de partida do trem t ∈ Tβ|j ∈ t
var_c_max = model.addVar(vtype=gp.GRB.CONTINUOUS, name="c_max")   ## Cmax = horário da partida do último vagão j ∈ t, t ∈ Tβ (makespan)

In [10]:
# Define objective function
model.setObjective(var_c_max, gp.GRB.MINIMIZE)

In [11]:
# Equation 2
for j in range(n):
  for i in range(o):
    if (i+1 < o):
      model.addConstr
      (
        var_y[i+1,j] 
        >= var_y[i,j] 
        + (P * gp.quicksum(var_z[i,q,j] for q in range(m)))
      )

In [12]:
# Equation 3
for j in range(n):
  for l in range(n):
    for i in range(o):
      for k in range(o):
        for q in range(m):
          if (i+1 < o):
            model.addConstr
            (
              var_y[i+1,j] 
              >= var_y[k,l]
              - (M * var_x[q,i,j,k,l])
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [13]:
# Equation 4
for j in range(n):
  for l in range(n):
    for i in range(o):
      for k in range(o):
        for q in range(m):
          if (i+1 < o):
            model.addConstr
            (
              var_y[k,l] 
              >= var_y[i+1,j]
              - (M * (1 - var_x[q,i,j,k,l]))
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [14]:
# Equation 5
for j in range(n):
  for i in range(o):
    if (i == 0):
      model.addConstr
      (
        gp.quicksum(var_z[i,q,j] for q in range(m))
        <= 1
      )

In [15]:
# Equation 6
for j in range(n):
  for l in range(n):
    for i in range(o): 
      for k in range(o):
        for q in range(m):
          if (k-1 > 0 and i+1 < o and j != l):
            model.addConstr
            (
              var_x[q,i,j,k,l] 
              + gp.quicksum(var_x[s,k-1,l,i+1,j] for s in range(m))
              <= 1
            )

In [16]:
# Equation 7
for j in range(n):
  for l in range(n):
    for i in range(o): 
      for k in range(o):
        for q in range(m):
          if (j != l):
            model.addConstr
            (
              var_x[q,i,j,k,l] 
              + var_x[q,k,l,i,j] 
              >= 1
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [17]:
# Equation 8
for j in range(n):
  for i in range(o):
    if (i-1 > 0):
      model.addConstr
      (
        gp.quicksum(var_z[i,q,j] for q in range(m))
        == 1
        - gp.quicksum(var_f[k,j] for k in range(i-1))
      )

In [18]:
# Equation 9
for j in t_beta:
  for i in range(o):
    model.addConstr(var_f[i,j] <= var_z[i,PSA[j],j])

In [19]:
# Equation 10
for j in t_beta:
  model.addConstr(gp.quicksum(var_f[i,j] for i in range(o)) == 1)

In [20]:
# Equation 11
for j in t_alfa:
  model.addConstr(gp.quicksum(var_f[i,j] for i in range(o)) == 0)

In [21]:
# Equation 12
for j in range(n):
  for i in range(o):
    for q in range(m):
      if (i == 0 and q == PCH[j]):
        model.addConstr(var_z[i,q,j] == 1)

In [22]:
# Equation 13
for j in t_beta:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_y[i,j] == RELEASE[j])

In [23]:
# Equation 14
for j in t_beta:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_y[i,j] >= RELEASE[j])

In [24]:
# Equation 15
for j in t_alfa:
  for l in t_alfa:
    for i in range(o):
      if (i == 0):
        model.addConstr(var_y[i,j] == var_y[i,l])

In [25]:
# Equation 16
for j in range(n):
  for i in range(o):
    for q in range(m):
      if (i+1 < o):
        model.addConstr(var_z[i+1,q,j] <= gp.quicksum(AMV[s][q] * var_z[i,s,j] for s in range(m)))

In [26]:
# Equation 17
for j in t_beta:
  for l in t_beta:
    for i in range(o):
      model.addConstr(var_c[j] >= var_y[i,l])

In [27]:
# Equation 18
for j in t_alfa:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_c[j] >= var_y[i,j])

In [28]:
# Equation 19
for j in range(n):
  for i in range(o):
    if (i+1 < o):
      model.addConstr
      (
        var_y[i+1,j]
        >= var_c[j] 
        - M 
        * (1 - gp.quicksum(var_f[k,j] for k in range(i)))
      )

In [29]:
# Equation 20
for j in t_beta:
  model.addConstr(var_c_max >= var_c[j])

In [30]:
# Execute model
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 276 rows, 5643 columns and 870 nonzeros
Model fingerprint: 0x97f69855
Variable types: 33 continuous, 5610 integer (5610 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 10.0000000
Presolve removed 276 rows and 5643 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


In [31]:
# Print the values of all variables
for v in model.getVars():
    print(f"{v.VarName} = {v.X}")

y[0,0] = 0.0
y[0,1] = 10.0
y[1,0] = 0.0
y[1,1] = 0.0
y[2,0] = 0.0
y[2,1] = 0.0
y[3,0] = 0.0
y[3,1] = 0.0
y[4,0] = 0.0
y[4,1] = 0.0
y[5,0] = 0.0
y[5,1] = 0.0
y[6,0] = 0.0
y[6,1] = 0.0
y[7,0] = 0.0
y[7,1] = 0.0
y[8,0] = 0.0
y[8,1] = 0.0
y[9,0] = 0.0
y[9,1] = 0.0
y[10,0] = 0.0
y[10,1] = 0.0
y[11,0] = 0.0
y[11,1] = 0.0
y[12,0] = 0.0
y[12,1] = 0.0
y[13,0] = 0.0
y[13,1] = 0.0
y[14,0] = 0.0
y[14,1] = 0.0
z[0,0,0] = 1.0
z[0,0,1] = 1.0
z[0,1,0] = 1.0
z[0,1,1] = 1.0
z[0,2,0] = 1.0
z[0,2,1] = 1.0
z[0,3,0] = 1.0
z[0,3,1] = 1.0
z[0,4,0] = 1.0
z[0,4,1] = 1.0
z[0,5,0] = 1.0
z[0,5,1] = 1.0
z[1,0,0] = 1.0
z[1,0,1] = 1.0
z[1,1,0] = 1.0
z[1,1,1] = 1.0
z[1,2,0] = 1.0
z[1,2,1] = 1.0
z[1,3,0] = 1.0
z[1,3,1] = 1.0
z[1,4,0] = 1.0
z[1,4,1] = 1.0
z[1,5,0] = 1.0
z[1,5,1] = 1.0
z[2,0,0] = 1.0
z[2,0,1] = 1.0
z[2,1,0] = 1.0
z[2,1,1] = 1.0
z[2,2,0] = 1.0
z[2,2,1] = 1.0
z[2,3,0] = 1.0
z[2,3,1] = 1.0
z[2,4,0] = 1.0
z[2,4,1] = 1.0
z[2,5,0] = 1.0
z[2,5,1] = 1.0
z[3,0,0] = 1.0
z[3,0,1] = 1.0
z[3,1,0] = 1.0
z[3,1,1] = 1.0